In [2]:
!python3 tools/track.py -f exps/example/mot/yolox_x_ablation.py -c pretrained/bytetrack_ablation.pth.tar -b 1 -d 1 --fp16 --fuse --matching reid

2024-05-10 15:35:56 | INFO     | __main__:160 - Args: Namespace(experiment_name='yolox_x_ablation', name=None, dist_backend='nccl', dist_url=None, batch_size=1, devices=1, local_rank=0, num_machines=1, machine_rank=0, exp_file='exps/example/mot/yolox_x_ablation.py', fp16=True, fuse=True, trt=False, test=False, speed=False, opts=[], ckpt='pretrained/bytetrack_ablation.pth.tar', conf=0.01, nms=0.7, tsize=None, seed=None, track_thresh=0.6, track_buffer=30, match_thresh=0.9, min_box_area=100, mot20=False, feat_model_name='resnet18', matching='reid')
/home/huythanhdang/.local/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2024-05-10 15:35:57 | INFO     | __main__:170 - Model Summary: Params: 99.00M, Gflops: 793.21
/home/huythanhdang/.local/l

In [1]:
import torch
from fastreid.config import get_cfg
from fastreid.modeling import build_model
import numpy as np
import cv2

cropped_image_np = np.load('/mnt/c/Users/NCPC/Desktop/thesis/ByteTrack/cropped_image.npy')
tensor = torch.from_numpy(cropped_image_np).float()
print(tensor.shape)
print(cropped_image_np)
# Chuẩn hóa tensor để các giá trị nằm trong khoảng 0-1
normalized_tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min())

# Chuyển đổi kích thước tensor
resized_tensor = torch.nn.functional.interpolate(normalized_tensor, size=(256, 128), mode='bicubic', align_corners=False)


torch.Size([1, 3, 59, 21])
[[[[-0.9365  -1.005   -1.005   ... -1.159   -1.296   -1.433  ]
   [-0.8677  -1.039   -1.021   ... -1.039   -1.364   -1.484  ]
   [-0.885   -0.9707  -0.9536  ... -1.107   -1.45    -1.501  ]
   ...
   [-0.628   -0.628   -0.628   ... -0.662   -0.645   -0.645  ]
   [-0.5938  -0.697   -0.697   ... -0.662   -0.645   -0.645  ]
   [-0.5767  -0.662   -0.662   ... -0.628   -0.645   -0.645  ]]

  [[-0.74    -0.81    -0.81    ... -1.003   -1.178   -1.318  ]
   [-0.6704  -0.845   -0.8276  ... -0.8804  -1.248   -1.37   ]
   [-0.6875  -0.758   -0.758   ... -0.968   -1.335   -1.388  ]
   ...
   [-0.4602  -0.4426  -0.4426  ... -0.4775  -0.4075  -0.4075 ]
   [-0.4075  -0.5127  -0.5127  ... -0.4602  -0.4075  -0.4075 ]
   [-0.3901  -0.4775  -0.4775  ... -0.4426  -0.4075  -0.4075 ]]

  [[-0.4624  -0.532   -0.532   ... -0.689   -0.881   -1.0205 ]
   [-0.3926  -0.567   -0.5493  ... -0.5845  -0.95    -1.072  ]
   [-0.4102  -0.4973  -0.4797  ... -0.6543  -1.037   -1.09   ]
   ...
   

In [2]:
# Đường dẫn đến file trọng số của mô hình FastReID đã được huấn luyện
model_weights = "/mnt/c/Users/NCPC/Desktop/thesis/ByteTrack/fast-reid/logs/market1501/bagtricks_R50/model_final.pth"

# Tạo một mô hình mới
cfg = get_cfg()
cfg.merge_from_file("/mnt/c/Users/NCPC/Desktop/thesis/ByteTrack/fast-reid/logs/market1501/bagtricks_R50/config.yaml")  # Thay đường dẫn với file cấu hình của mô hình
model = build_model(cfg)

# Load trọng số cho mô hình
checkpoint = torch.load(model_weights, map_location=torch.device("cpu"))
model.load_state_dict(checkpoint["model"])

# Đặt mô hình vào chế độ đánh giá
model.eval()

# Tensor đại diện cho ảnh đã cắt từ bounding box, bạn cần điều chỉnh theo kích thước và định dạng của ảnh đầu vào của mô hình
# cropped_image_tensor = torch.from_numpy(resized_tensor)
resized_tensor = resized_tensor.cuda()
print(resized_tensor.shape)
print(type(resized_tensor))

# Trích xuất đặc trưng từ tensor ảnh đã cắt
with torch.no_grad():
    features = model.backbone(resized_tensor)  # Trích xuất đặc trưng

# In ra kích thước của vector đặc trưng
print("Feature shape:", features.shape)

torch.Size([1, 3, 256, 128])
<class 'torch.Tensor'>
Feature shape: torch.Size([1, 2048, 16, 8])


In [4]:
import torch.onnx

# Tên tệp ONNX đầu ra
onnx_filename = "fastreid_model.onnx"

# Xác định kích thước đầu vào dựa trên cấu hình của mô hình
input_height = 256
input_width = 128
input_channels = 3  # 3 kênh cho ảnh màu RGB

# Tạo một tensor ngẫu nhiên làm đầu vào cho việc xuất ONNX
dummy_input = torch.randn(1, input_channels, input_height, input_width).cuda()
input_names = ["input"]

# Xác định các tên đầu ra mà bạn muốn giữ lại trong tệp ONNX, ở đây tôi giả sử là "output"
output_names = ["output"]

# Export mô hình sang ONNX format
torch.onnx.export(model,                                # Mô hình đã được tạo
                  dummy_input,       # Giả lập một đầu vào (cần được đưa vào GPU nếu có)
                  onnx_filename,                        # Tên tệp ONNX đầu ra
                  input_names=input_names,              # Tên của các đầu vào
                  output_names=output_names,            # Tên của các đầu ra
                  opset_version=11,                     # Phiên bản ONNX
                  export_params=True)                   # Xuất các trọng số của mô hình

print("ONNX file exported successfully.")


/home/huythanhdang/.local/lib/python3.10/site-packages/torch/onnx/utils.py:617: UserWarning: ONNX Preprocess - Removing mutation from node aten::sub_ on block input: 'batched_inputs.1'. This changes graph semantics. (Triggered internally at ../torch/csrc/jit/passes/onnx/remove_inplace_ops_for_onnx.cpp:335.)
  _C._jit_pass_onnx_remove_inplace_ops_for_onnx(graph, module)


ONNX file exported successfully.


In [8]:
import torch
from torchvision import transforms
from PIL import Image

# Đường dẫn của hình ảnh
image_path = "datasets/mot/train/MOT17-02-FRCNN/img1/000066.jpg"

# Đọc hình ảnh và chuyển đổi thành tensor
image = Image.open(image_path)

# Sử dụng transforms để chuyển đổi hình ảnh thành tensor
preprocess = transforms.Compose([
    transforms.ToTensor(),
])

# Áp dụng các phép biến đổi cho hình ảnh
image_tensor = preprocess(image)

# Chuyển đổi dtype của tensor thành torch.float16
image_tensor


tensor([[[0.2118, 0.2118, 0.2118,  ..., 0.3804, 0.3804, 0.3804],
         [0.2118, 0.2118, 0.2118,  ..., 0.3804, 0.3804, 0.3804],
         [0.2118, 0.2118, 0.2118,  ..., 0.3804, 0.3804, 0.3804],
         ...,
         [0.2471, 0.2392, 0.2275,  ..., 0.2118, 0.2118, 0.2118],
         [0.2392, 0.2353, 0.2235,  ..., 0.2118, 0.2118, 0.2118],
         [0.2314, 0.2275, 0.2275,  ..., 0.2118, 0.2118, 0.2118]],

        [[0.2196, 0.2196, 0.2196,  ..., 0.3686, 0.3686, 0.3686],
         [0.2196, 0.2196, 0.2196,  ..., 0.3686, 0.3686, 0.3686],
         [0.2196, 0.2196, 0.2196,  ..., 0.3686, 0.3686, 0.3686],
         ...,
         [0.2588, 0.2510, 0.2392,  ..., 0.2235, 0.2235, 0.2235],
         [0.2510, 0.2471, 0.2353,  ..., 0.2235, 0.2235, 0.2235],
         [0.2431, 0.2392, 0.2392,  ..., 0.2235, 0.2235, 0.2235]],

        [[0.1686, 0.1686, 0.1686,  ..., 0.4118, 0.4118, 0.4118],
         [0.1686, 0.1686, 0.1686,  ..., 0.4118, 0.4118, 0.4118],
         [0.1686, 0.1686, 0.1686,  ..., 0.4118, 0.4118, 0.

In [11]:
# Trích xuất đặc trưng từ tensor ảnh đã cắt
with torch.no_grad():
    features_62 = model.backbone(image_tensor.cuda().unsqueeze(0))  # Trích xuất đặc trưng

In [12]:
features_62.shape

torch.Size([1, 2048, 68, 120])

In [13]:
image_tensor.shape

torch.Size([3, 1080, 1920])